In [3]:
import os
os.chdir('/Users/roberttiu/Desktop/RecSys_Course_AT_PoliMi-master')

from Evaluation.Evaluator import EvaluatorHoldout
import numpy
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
#from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.load_ICM import load_ICM 
from Utils.load_URM import load_URM
from scipy import sparse as sps
import pandas as pd
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization
import json
# import tables

In [4]:
def combine(ICM: sps.csr_matrix, URM : sps.csr_matrix):
    return sps.hstack((URM.T, ICM), format='csr')


URM_all = load_URM('new_in/data_train.csv')
ICM_channel_all = load_ICM('new_in/data_ICM_channel.csv')
ICM_event_all = load_ICM('new_in/data_ICM_event.csv')
ICM_genre_all = load_ICM('new_in/data_ICM_genre.csv')
ICM_subgenre_all = load_ICM('new_in/data_ICM_subgenre.csv')
ICM_all = sps.hstack((ICM_channel_all, ICM_event_all, ICM_genre_all, ICM_subgenre_all))
URM_ICM_all = combine(ICM=ICM_all, URM=URM_all)

print(URM_ICM_all)

  (0, 1445)	1.0
  (0, 1912)	1.0
  (0, 1932)	1.0
  (0, 4043)	1.0
  (0, 5104)	1.0
  (0, 6696)	1.0
  (0, 10204)	1.0
  (0, 10351)	1.0
  (0, 10817)	1.0
  (0, 11002)	1.0
  (0, 11147)	1.0
  (0, 11386)	1.0
  (0, 11717)	1.0
  (0, 11720)	1.0
  (0, 13673)	1.0
  (0, 135644)	1.0
  (0, 371936)	1.0
  (0, 372011)	1.0
  (1, 92)	1.0
  (1, 288)	1.0
  (1, 325)	1.0
  (1, 493)	1.0
  (1, 669)	1.0
  (1, 959)	1.0
  (1, 1018)	1.0
  :	:
  (18058, 11065)	1.0
  (18058, 11191)	1.0
  (18058, 11256)	1.0
  (18058, 11257)	1.0
  (18058, 11352)	1.0
  (18058, 11463)	1.0
  (18058, 11504)	1.0
  (18058, 11745)	1.0
  (18058, 11892)	1.0
  (18058, 11954)	1.0
  (18058, 12016)	1.0
  (18058, 12161)	1.0
  (18058, 12244)	1.0
  (18058, 12396)	1.0
  (18058, 12583)	1.0
  (18058, 12699)	1.0
  (18058, 12768)	1.0
  (18058, 13098)	1.0
  (18058, 13194)	1.0
  (18058, 13389)	1.0
  (18058, 13836)	1.0
  (18058, 110183)	1.0
  (18058, 190612)	1.0
  (18058, 371936)	1.0
  (18058, 371979)	1.0


In [5]:

recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_ICM_all.T, verbose=True)
# Starting hyperparameters from Sanvito repo
''' PARAMETRI DI SANVITO STANDARD
SLIM_recommender.fit(
        alpha=0.00026894910579512645,
        l1_ratio=0.08074126876487486,
        topK=int(395.376118479588),
        workers=6
    )
'''

''' PARAMETRI CON CUI HO FATTO LA SUBMISSION DA 0.47699
recommender.fit(
        alpha=0.00027153127362632663,
        l1_ratio=0.07989965746223235,
        topK=int(397.147437437882),
        workers=6
    )
'''

''' PARAMETRI DELLA SUBMISSION DA 0.40 CIRCA

recommender.fit(
        alpha=0.007596772625600448,
        l1_ratio=0.09354638545594608,
        topK=410,
        workers=6
    )
    
'''

recommender.fit(
        alpha=0.00027153127362632663,
        l1_ratio=0.07989965746223235,
        topK=int(397.147437437882),
        workers=6
    )




SLIMElasticNetRecommender: URM Detected 10245 ( 2.8%) users with no interactions.


100%|████████████████████████████████████▉| 18032/18059 [17:43<00:01, 21.93it/s]

In [6]:
from Evaluation.Evaluator import EvaluatorHoldout

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, result_string = evaluator_validation.evaluateRecommender(recommender_ItemKNNCF)

result_string, result_df

EvaluatorHoldout: Ignoring 13640 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


NameError: name 'recommender_ItemKNNCF' is not defined

100%|████████████████████████████████████▉| 18056/18059 [18:00<00:00, 21.93it/s]

In [4]:
recommender.URM_train = URM_all.tocsr()


users_to_recommend = pd.read_csv('new_in/data_target_users_test.csv', names = ['user_id'], header = 0)

submission = []

for user in users_to_recommend["user_id"].values:
    submission.append((user, recommender.recommend(user, 10)))
    

def write_submission(submissions):
    with open('./submission_post_hypertuning_101221.csv', 'w') as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

submission



100%|████████████████████████████████████▉| 18056/18059 [05:30<00:00, 54.69it/s]


[(0, [3899, 16365, 8635, 640, 5433, 2665, 1137, 9131, 1792, 17961]),
 (1, [3473, 940, 14400, 640, 16012, 15119, 16365, 14358, 13032, 2665]),
 (2, [6177, 1551, 17022, 17330, 5925, 15119, 16365, 2748, 2665, 13134]),
 (3, [8194, 15551, 16359, 15082, 960, 5885, 5981, 15844, 15428, 7870]),
 (4, [6177, 8194, 3473, 9871, 6011, 5299, 1259, 7295, 2665, 2152]),
 (5, [1846, 16640, 9621, 14396, 7867, 11227, 202, 13098, 13585, 8852]),
 (6, [6214, 985, 14358, 16846, 8207, 14181, 14868, 5273, 13134, 12792]),
 (7, [15119, 13963, 4292, 640, 3768, 17364, 3433, 2886, 3899, 9131]),
 (8, [8635, 2665, 1259, 5605, 10865, 3619, 3768, 4853, 5299, 8502]),
 (9, [5605, 1614, 14181, 3473, 8635, 1551, 8207, 13318, 6214, 15119]),
 (10, [3899, 14400, 6214, 6975, 5273, 16365, 14358, 13229, 16012, 16846]),
 (11, [14181, 3473, 10865, 11359, 10370, 249, 13134, 12390, 7295, 7309]),
 (12, [3899, 3200, 14358, 1137, 6724, 15778, 13134, 1792, 6011, 5925]),
 (13, [8635, 6214, 15119, 14358, 13178, 5299, 6975, 5403, 15778, 2886]

In [56]:
string = ' '.join([str(item)for item in submission[0][1]])
string

'17438 8719 11300 3713 12620 10993 15932 1734 2922 15361'